<a href="https://colab.research.google.com/github/Ramjeet-Dixit/IITM-AIML-Rdixit/blob/main/MFNN_in_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using NN in Machine Learning Tasks

1. Images -- Image classification

2. Videos

3. Text

4. Sounds

We can also apply MFNN in Regression and classification

**Applying a Multi-Feed Neural Network (MFNN) for Regression**

A Multi-Feed Neural Network (MFNN) is simply a neural architecture where multiple inputs (or input representations) are fed into the model through separate branches, processed independently, and then merged before prediction.

This is extremely useful in regression tasks when your dataset contains heterogeneous inputs, such as:

Numeric features

Categorical embeddings

Engineered features

Image + tabular

Time-series + static data

The MFNN allows you to build specialized sub-networks for each input.

Input A → Dense layers → Feature Vector A

Input B → Dense layers → Feature Vector B

Input C → Dense layers → Feature Vector C

Merged Vector → Regression Head → Output


Predict house prices using:

Numerical features → (Area, Bedrooms, Age, Distance to metro…)

Categorical features → (City, Zone)

Neighbourhood statistics → (crime index, greenery score)

Each group is very different → MFNN is ideal.

**Implement MFNN in regression using California Housing Dataset**

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

data = pd.read_csv("/content/mydata.xls")

data.head()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras import layers, Model

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

df = pd.read_csv("/content/mydata.xls")

# Separate features and target
X = df.drop("MedHouseVal", axis=1)
y = df["MedHouseVal"]

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Show first few rows
X.head()


Shape of X: (20640, 8)
Shape of y: (20640,)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


## 3. Feature Grouping for Multi-Feed Architecture

We will create **three logical feature groups**:

### 🧩 Group A – Socioeconomic / House Internals
- `MedInc`   (Median income)
- `AveRooms` (Average rooms)
- `AveBedrms` (Average bedrooms)
- `AveOccup` (Average occupants per household)

These features describe economic strength and internal house attributes.

---

### 📍 Group B – Location Features
- `Latitude`
- `Longitude`

Location has a strong, non-linear relationship with house prices and deserves its own branch.

---

### 👥 Group C – Demographic / Age Features
- `HouseAge`
- `Population`

These features describe age of homes and area population.


In [ ]:
# Define feature groups
group_a = X[["MedInc", "AveRooms", "AveBedrms", "AveOccup"]] #list of lists: selecting relevant columns from dataframe
group_b = X[["Latitude", "Longitude"]]
group_c = X[["HouseAge", "Population"]]

print("Group A shape:", group_a.shape)
print("Group B shape:", group_b.shape)
print("Group C shape:", group_c.shape)


Group A shape: (20640, 4)
Group B shape: (20640, 2)
Group C shape: (20640, 2)


## 4. Train–Test Split and Scaling

We will:
- Split into **train** and **test** sets (80/20)
- Use **separate scalers** for each feature group  
  (because they go into different branches of the network)

Note: Scaling is important for neural networks to train stably.


In [ ]:
# Train-test split for all groups simultaneously
(
    X_train_a, X_test_a,
    X_train_b, X_test_b,
    X_train_c, X_test_c,
    y_train, y_test
) = train_test_split(
    group_a, group_b, group_c, y,
    test_size=0.2,
    random_state=42
)

# Initialize scalers for each group
scaler_a = StandardScaler()
scaler_b = StandardScaler()
scaler_c = StandardScaler()

# Fit on train, transform train & test
X_train_a = scaler_a.fit_transform(X_train_a)
X_test_a = scaler_a.transform(X_test_a)

X_train_b = scaler_b.fit_transform(X_train_b)
X_test_b = scaler_b.transform(X_test_b)

X_train_c = scaler_c.fit_transform(X_train_c)
X_test_c = scaler_c.transform(X_test_c)

# Convert targets to numpy arrays
y_train = y_train.values
y_test = y_test.values

print("Train shapes:", X_train_a.shape, X_train_b.shape, X_train_c.shape, y_train.shape)
print("Test shapes:", X_test_a.shape, X_test_b.shape, X_test_c.shape, y_test.shape)


Train shapes: (16512, 4) (16512, 2) (16512, 2) (16512,)
Test shapes: (4128, 4) (4128, 2) (4128, 2) (4128,)


## 5. Build the Multi-Feed Neural Network (MFNN)

We will design **three branches**:

### 🔹 Branch A – Socioeconomic / House Internals
- Input size: 4
- Dense(64) → Dense(32)

### 🔹 Branch B – Location Features
- Input size: 2
- Dense(32) → Dense(16)

### 🔹 Branch C – Demographic / Age Features
- Input size: 2
- Dense(32) → Dense(16)

Then we will:
- Concatenate all three branch outputs
- Add two more Dense layers
- Output a single neuron for **regression** (house value)

This structure allows each feature group to learn its own representation before joining forces.


In [ ]:
# ----- Branch A: Socioeconomic / Internals -----
input_a = layers.Input(shape=(4,), name="socioeconomic_input") #input layer
a = layers.Dense(64, activation="relu", name="a_dense_1")(input_a)
a = layers.Dense(32, activation="relu", name="a_dense_2")(a)